In [1]:
#limpio la memoria
rm( list=ls() )  #remove all objects
gc()             #garbage collection

require("data.table")
require("scales")
require("lightgbm")

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,604568,32.3,1292120,69.1,1292120,69.1
Vcells,1087066,8.3,8388608,64.0,1632674,12.5


Loading required package: data.table

Loading required package: scales

Loading required package: lightgbm

Loading required package: R6



In [2]:
#Parametros del script
setwd("C:/_MCD/Labo1/code")  #Establezco el Working Directory
#setwd("~/buckets/b1/")

In [4]:
graficar_campo  <- function( campo, periodos_analisis,dataset )
{

  #quito de grafico las colas del 5% de las densidades

  tb_quantiles  <- data.table( qmin= numeric(),  qmax= numeric() )

  for( periodo in periodos_analisis )
  {
    qu  <- quantile(  dataset[ foto_mes==periodo , get(campo) ] , prob= c(0.05, 0.95), na.rm=TRUE )
    tb_quantiles  <- rbind( tb_quantiles, use.names=FALSE, as.list(qu) )
  }

  xxmin  <- tb_quantiles[ , min( qmin ) ]
  xxmax  <- tb_quantiles[ , max( qmax ) ]

  yymax  <- 0
  for( per in periodos_analisis )
  {
    den  <- density( dataset[ foto_mes==per, get(campo) ],
                     kernel="gaussian", na.rm=TRUE )

    mayor  <- max( den$y )
    if( mayor > yymax ) yymax <- mayor 
  }

  densidad_A  <- density( dataset[ foto_mes==periodos_analisis[1], get(campo) ],
                          kernel="gaussian", na.rm=TRUE )


  plot( densidad_A,
        col= GLOBAL_colores[1],
        xlim= c( xxmin, xxmax ),
        ylim= c( 0, yymax ),
        main= paste0( campo  ) 
      )

  for( per in 2:length(periodos_analisis) )
  {
    densidad_B  <- density( dataset[ foto_mes==periodos_analisis[ per ], get(campo) ],
                            kernel="gaussian", na.rm=TRUE )

    lines(densidad_B, col= GLOBAL_colores[per], lty=1)
  }
  
  legend(  "topright",  
           legend= periodos_analisis,
           col=GLOBAL_colores, lty=1 )

}

In [5]:
generarPdf <- function(nombrePdf,dataset)
{
    #creo la clase_binaria SI={ BAJA+1, BAJA+2 }    NO={ CONTINUA }
    dataset[ foto_mes<=202103, 
             clase01 :=  ifelse( clase_ternaria=="CONTINUA", 0, 1 ) ]


    campos_buenos  <- setdiff( colnames(dataset), c("clase_ternaria","clase01") )

    #dejo los datos en el formato que necesita LightGBM
    dtrain  <- lgb.Dataset( data= data.matrix(  dataset[ foto_mes %in% c( 202101), campos_buenos, with=FALSE]),
                        label= dataset[foto_mes %in% c( 202101), clase01] )

    #genero el modelo
    #estos hiperparametros  salieron de una laaarga Optmizacion Bayesiana
    modelo  <- lgb.train( data= dtrain,
                      param= list( objective=          "binary",
                                   max_bin=             31,
                                   learning_rate=        0.018,
                                   num_iterations=     526,
                                   num_leaves=        1608,
                                   min_data_in_leaf=  5200,
                                   feature_fraction=     0.6,
                                   seed=              102191
                                  )
                    )

    tb_importancia  <-  as.data.table( lgb.importance(modelo) ) 
    campos_ordenados  <- c( tb_importancia$Feature,  setdiff( colnames(dataset), tb_importancia$Feature ) )
    campos_ordenados  <-  setdiff(  campos_ordenados,  c( "foto_mes","clase_ternaria","clase01" ) )
   
    dataset[  , foto_mes := as.character( foto_mes ) ]

    pdf(nombrePdf)

    for( campo in  campos_ordenados )
    {
      cat( campo, "  " )

      graficar_campo( campo, periodos_analisis,dataset )
    }

    dev.off()
}

In [6]:
periodos_analisis  <- c( 202101, 202103 )
GLOBAL_colores <-  viridis_pal()(length( periodos_analisis ) )

In [7]:
dataset  <- fread("./datasets/comp1/competencia1_2022_fe_inicial_deflacion.csv")  
generarPdf("./exp/comp1-EDA/densidad_deflacion.pdf",dataset)

[LightGBM] [Info] Number of positive: 1514, number of negative: 159828
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.028777 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3907
[LightGBM] [Info] Number of data points in the train set: 161342, number of used features: 158
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009384 -> initscore=-4.659343
[LightGBM] [Info] Start training from score -4.659343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

png 
  2

In [8]:
dataset  <- fread("./datasets/comp1/competencia1_2022_fe_inicial_rank0simple.csv")  
generarPdf("./exp/comp1-EDA/densidad_rank0.pdf",dataset)

[LightGBM] [Info] Number of positive: 1514, number of negative: 159828
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.172249 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4481
[LightGBM] [Info] Number of data points in the train set: 161342, number of used features: 174
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009384 -> initscore=-4.659343
[LightGBM] [Info] Start training from score -4.659343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positiv

png 
  2

In [9]:
dataset  <- fread("./datasets/comp1/competencia1_2022_fe_inicial_rank0fijo.csv")  
generarPdf("./exp/comp1-EDA/densidad_rank0fijo.pdf",dataset)

[LightGBM] [Info] Number of positive: 1514, number of negative: 159828
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.042404 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3934
[LightGBM] [Info] Number of data points in the train set: 161342, number of used features: 158
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.009384 -> initscore=-4.659343
[LightGBM] [Info] Start training from score -4.659343
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

png 
  2